### Parse gkm-SVM explain output

In [ ]:

#! pip install numpy==1.9 # failed
#! pip install numpy==1.24.3
#! pip install biopython=1.81
#! pip install deeplift=0.6.13.0
#! pip install modisco==0.5.16.0
# # Successfully installed contourpy-1.1.1 cycler-0.12.1 fonttools-4.43.1 h5py-3.10.0 igraph-0.10.8 importlib-resources-6.1.0 joblib-1.3.2 kiwisolver-1.4.5 leidenalg-0.10.1 matplotlib-3.8.1 modisco-0.5.16.0 packaging-23.2 pillow-10.1.0 psutil-5.9.6 pyparsing-3.1.1 python-dateutil-2.8.2 scikit-learn-1.3.2 scipy-1.11.3 six-1.16.0 texttable-1.7.0 threadpoolctl-3.2.0 tqdm-4.66.1 zipp-3.17.0
#! pip install tensorflow==2.13.0
#! pip install ipykernel==6.26.0

In [ ]:
conda activate tfmodisco

Helper functions

In [1]:
import numpy as np

def one_hot_encode_along_channel_axis(sequence):
    to_return = np.zeros((len(sequence),4), dtype=np.int8)
    seq_to_one_hot_fill_in_array(zeros_array=to_return,
                                 sequence=sequence, one_hot_axis=1)
    return to_return

def seq_to_one_hot_fill_in_array(zeros_array, sequence, one_hot_axis):
    assert one_hot_axis==0 or one_hot_axis==1
    if (one_hot_axis==0):
        assert zeros_array.shape[1] == len(sequence)
    elif (one_hot_axis==1): 
        assert zeros_array.shape[0] == len(sequence)
    #will mutate zeros_array
    for (i,char) in enumerate(sequence):
        if (char=="A" or char=="a"):
            char_idx = 0
        elif (char=="C" or char=="c"):
            char_idx = 1
        elif (char=="G" or char=="g"):
            char_idx = 2
        elif (char=="T" or char=="t"):
            char_idx = 3
        elif (char=="N" or char=="n"):
            continue #leave that pos as all 0's
        else:
            raise RuntimeError("Unsupported character: "+str(char))
        if (one_hot_axis==0):
            zeros_array[char_idx,i] = 1
        elif (one_hot_axis==1):
            zeros_array[i,char_idx] = 1

def normalize_scores(impscores, hyp_impscores, onehot_data):
  #normalize the hyp scores such that, at each position, hypothetical importance
  # scores that have the same sign as the original importance score all sum
  # up to the original importance score value. The rationale is that if
  # multiple different bases at a position could produce a similar score,
  # the specific identity of each individual base is less important.
  #Empirically, hypothetical scores like these appear to work better for
  # motif discovery. Using normalized importance scores derived by taking
  # the elementwise product of the normalized hypothetical scores and
  # the one-hot encoding also seems to reduce noise.
  normed_hyp_impscores = []
  normed_impscores = []
  for i in range(len(impscores)):
      imp_score_each_pos = np.sum(impscores[i],axis=-1)
      imp_score_sign_each_pos = np.sign(imp_score_each_pos)
      hyp_scores_same_sign_mask = (np.sign(hyp_impscores[i])
                                   *imp_score_sign_each_pos[:,None] > 0)
      hyp_scores_same_sign_imp_scores_sum = np.sum(
          hyp_impscores[i]*hyp_scores_same_sign_mask,axis=-1)
      norm_ratio = imp_score_each_pos/hyp_scores_same_sign_imp_scores_sum
      norm_hyp = hyp_impscores[i]*norm_ratio[:,None]
      normed_hyp_impscores.append(norm_hyp)
      normed_impscores.append(norm_hyp*onehot_data[i])
  return normed_impscores, normed_hyp_impscores


Import FASTA sequences used with `gkmexplain`, importance scores and hypothetical importance scores, and save input arrays for TFMoDisco

In [ ]:
import os

dir = "/home/anamaria/cluster/aelek/proj/scATAC_nvec_v2/clustering/SEACells/Results/ArchRProj_Nvec_TSS4_frag200/ArchRProj/"
dir_gkm = dir+"gkmSVM/PeakDifferential_cell_type/03/"
dir_fig = dir+"Plots/08_gkmSVM/PeakDifferential_cell_type/03/gkmekplain"
dir_mod = dir+"tfmodisco/"


explain_files = [filename for filename in os.listdir(dir_gkm) if filename.endswith('.explain.fasta')]
models = [x.replace('.explain.fasta', '') for x in explain_files]
models = sorted(models)
models

In [ ]:
import os
from Bio import SeqIO

for model in models:
    print(f"Model: {model}")
    # Read fasta sequences
    fasta_seqs=[]
    # with open(dir_gkm+model+".explain.fasta") as handle:
    with open(dir_gkm+"all.fasta") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            fasta_seqs.append(str(record.seq))
    # Filter out any sequences that contain 'N's
    onehot_data = [np.array(one_hot_encode_along_channel_axis(x)) for x in fasta_seqs] #  if ('N' not in x)
    print(f"\tNum onehot sequences: {len(onehot_data)}")
    # Read in the importance scores and hypothetical importance scores
    hyp_impscores = [w[0] for w in zip([
        np.array( [[float(z) for z in y.split(",")]
                    for y in x.rstrip().split("\t")[2].split(";")])
        for x in open(dir_gkm+model+".explain.hyp.txt")
    ],fasta_seqs)]# if 'N' not in w[1]]
    # Read in the hypothetical importance scores
    impscores = [w[0] for w in zip([
        np.array( [[float(z) for z in y.split(",")]
                    for y in x.rstrip().split("\t")[2].split(";")])
        for x in open(dir_gkm+model+".explain.txt")
    ],fasta_seqs)]# if 'N' not in w[1]]
    # Perform a sanity check to make sure that the importance score are the same as the
    # hypothetical scores multiplied elementwise with the one-hot encoding
    #assert (np.max([np.max(np.abs(z*y - x))
    #                for x,y,z in zip(impscores,
    #                                onehot_data,
    #                                hyp_impscores)]))==0
    # Save importance scores
    impscores = [x[0:250] for x in impscores]
    impscores_arr = np.array(impscores)
    impscores_arr = np.swapaxes(impscores_arr, 1, 2)
    np.savez(os.path.join(dir_mod, model+"_impscore.npz"), impscores_arr)
    # Save hypothetical importance scores
    hyp_impscores = [x[0:250] for x in hyp_impscores]
    hyp_impscores_arr = np.array(hyp_impscores)
    hyp_impscores_arr = np.swapaxes(hyp_impscores_arr, 1, 2)
    np.savez(os.path.join(dir_mod, model+"_hyp_impscores.npz"), hyp_impscores_arr)
    # Save one-hot data
    onehot_data = [x[0:250] for x in onehot_data]
    onehot_data_arr = np.array(onehot_data)
    onehot_data_arr = np.swapaxes(onehot_data_arr, 1, 2)
    np.savez(os.path.join(dir_mod, model+"_onehot_data.npz"), onehot_data_arr)
np.savez(os.path.join(dir_mod, "onehot_data.npz"), onehot_data_arr)

Normalize scores

In [ ]:
normed_impscores, normed_hyp_impscores = normalize_scores(
impscores=impscores, hyp_impscores=hyp_impscores, onehot_data=onehot_data)

# Normalize importance scores
imp_dir=os.path.join(dir, model+"_norm_impscores")
os.makedirs(imp_dir, exist_ok=True)

for i, peak in enumerate(normed_impscores):
    peak_arr = np.array(peak)
    fn = os.path.join(imp_dir,  f"norm_impscores_{i}.txt")
    np.savetxt(fn, peak_arr, delimiter="\t")

for i, peak in enumerate(normed_hyp_impscores):
    peak_arr = np.array(peak)
    fn = os.path.join(imp_dir,  f"norm_hyp_impscores_{i}.txt")
    np.savetxt(fn, peak_arr, delimiter="\t")

Visualize a few sequences as a sanity check

In [ ]:
from modisco.visualization import viz_sequence
from matplotlib import pyplot as plt
plt.style.use('default')

def plot_weights_pdf(fn, array,
                     figsize=(20,2),
                     **kwargs):
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111) 
    viz_sequence.plot_weights_given_ax(ax=ax, array=array,**kwargs)
    plt.savefig(fn)

sorted_indices = [x[0] for x in
                  sorted(enumerate([np.sum(x) for x in impscores]),
                         key=lambda x: -x[1])]

for idx in sorted_indices[0:10]:
    plot_weights_pdf(dir_fig+model+"_weights_"+str(idx)+".pdf", normed_impscores[idx], subticks_frequency=10)

### Run tfmodisco-lite from CL

In [ ]:
attr = np.load(os.path.join(dir_mod, model+"_impscore.npz"))
ohe = np.load(os.path.join(dir_mod, model+"_onehot_data.npz"))

for key in attr.files:
    print(f"The shape of the attribution scores is {attr[key].shape}")

for key in ohe.files:
    print(f"The shape of the one-hot encoding is {ohe[key].shape}")
    

In [ ]:
work_dir="/home/anamaria/cluster/aelek/proj/scATAC_nvec_v2/clustering/SEACells/"
dir_gkm=${work_dir}"Results/ArchRProj_Nvec_TSS4_frag200/ArchRProj/gkmSVM/PeakDifferential_cell_type/03/"
dir_mod=${work_dir}"Results/ArchRProj_Nvec_TSS4_frag200/ArchRProj/tfmodisco/"

#model="adult_cnidocyte"
model="adult_neuron_GATA_Islet_1" # problem with ARCH513
atr=${model}"_hyp_impscores.npz"
ohe=${model}"_onehot_data.npz"
res=${model}"_modisco_hyp.h5"
modisco motifs -s ${dir_mod}"/"${ohe} -a ${dir_mod}"/"${atr} -n 4000 -w 124 -o ${dir_mod}"/"${res}

# archetypes
mot_dir=${work_dir}"/Results/ArchRProj_Nvec_TSS4_frag200/ArchRProj/Archetypes/"
mot_fn="motif-archetypes-PPM-PCCnorm-0.8-IC0.5-8bp-pwms-tfs.meme"
ln -s ${mot_dir}"/"${mot_fn} ${dir_mod}"/"${mot_fn}
rep=${model}"_modisco_hyp_report"

# jaspar motifs
mot_fn="JASPAR2020_CORE_non-redundant_pfms_meme.txt"
rep=${model}"_modisco_hyp_jaspar_report"

# homer motifs
mot_fn="homer.motifs.meme"
rep=${model}"_modisco_hyp_homer_report"

# report
modisco report -i ${dir_mod}"/"${res} -o${dir_mod}"/"${rep} -s "" -m ${dir_mod}"/"${mot_fn} -n 3

### Run modified tfmodisco-lite workflow

Save importance scores of all models for all sequences into an Ordered Dictionary

In [ ]:
from Bio import SeqIO
import os

import h5py
import modisco.util


dir="/home/anamaria/cluster/aelek/proj/scATAC_nvec_v2/clustering/SEACells/Results/ArchRProj_Nvec_TSS4_frag200/ArchRProj/"
dir_gkm = dir+"gkmSVM/PeakDifferential_cell_type/03/"
dir_fig = dir+"Plots/08_gkmSVM/PeakDifferential_cell_type/03/tfmodisco/"
dir_mod = dir+"tfmodisco/"

#explain_files = [filename for filename in os.listdir(dir_gkm) if filename.endswith('.explain.fasta')]
#models = [x.replace('.explain.fasta', '') for x in explain_files]
#models = sorted(models)

npz_files = [filename for filename in os.listdir(dir_mod) if filename.endswith('_impscore.npz')]
models = [x.replace('_impscore.npz', '') for x in npz_files]
models = sorted(models)

import numpy as np
from collections import OrderedDict

task_to_hyp_scores = OrderedDict()

for model_id, model in enumerate(models):
    print(f"Model: {model}")
    attr = np.load(os.path.join(dir_mod, model+"_impscore.npz"))
    attr = attr[attr.files[0]] 
    print(f"{attr.shape[0]} importance scores")
    task_to_hyp_scores[model] = []
    for seq_id in range(len(attr)):
        task_to_hyp_scores[model].append(np.transpose(attr[seq_id]))

# save
import pickle
pickle.dump(
    task_to_hyp_scores,
    open(os.path.join(dir_mod, "task_to_hyp_scores.pkl"), "wb"))

Params

In [ ]:
n_top_regions = 3_000
params = dict(
    sliding_window_size=15,
    flank_size=5,
    min_metacluster_size=100,
    weak_threshold_for_counting_sign=0.8,
    target_seqlet_fdr=0.2,
    min_passing_windows_frac=0.03,
    max_passing_windows_frac=0.2,
    n_leiden_runs=50,
    n_leiden_iterations=-1,
    min_overlap_while_sliding=0.7,
    nearest_neighbors_to_compute=500,
    affmat_correlation_threshold=0.15,
    tsne_perplexity=10.0,
    frac_support_to_trim_to=0.2,
    min_num_to_trim_to=30,
    trim_to_window_size=12,
    initial_flank_to_add=5,
    prob_and_pertrack_sim_merge_thresholds=[(0.8,0.8), (0.5, 0.85), (0.2, 0.9)],
    prob_and_pertrack_sim_dealbreaker_thresholds=[(0.4, 0.75), (0.2,0.8), (0.1, 0.85), (0.0,0.9)],
    subcluster_perplexity=50,
    merging_max_seqlets_subsample=300,
    final_min_cluster_size=10,
    min_ic_in_window=0.6,
    min_ic_windowsize=6,
    ppm_pseudocount=0.001,
    number_of_seqlets_to_sample = n_top_regions * 4)


Custom modisco-lite workflow by Seppe (run this from `~/lcb/aelek/gkmsvm_nvec/`)

In [ ]:
source /staging/leuven/stg_00002/lcb/sdewin/Programs/anaconda3/etc/profile.d/conda.sh
conda activate tfmodsico-lite

1. Get pattern for all cell types (tasks)

In [ ]:
import os
import pickle
with open(os.path.join("task_to_hyp_scores.pkl"), "rb") as f:
    task_to_hyp_scores = pickle.load(f)

tasks = list(task_to_hyp_scores.keys())

import numpy as np
seq_onehots_f = np.load("onehot_data.npz")
seq_onehots = seq_onehots_f['arr_0']
seq_onehots = np.transpose(seq_onehots_f['arr_0'], (0, 2, 1))

import sys
sys.path.insert(0, "/staging/leuven/stg_00002/lcb/sdewin/PhD/python_modules/custom_modisco/src")
from custom_modisco import workflow

# get patterns
topics_to_patterns = workflow.get_patterns_for_all_tasks(
    tasks = tasks,
    one_hot = seq_onehots,
    task_to_hyp_scores = task_to_hyp_scores,
    n_cpu = 8
)

# save
pickle.dump(
    topics_to_patterns,
    open(os.path.join("topics_to_patterns.pkl"), "wb"))

# ping me
import os
import requests
TOKEN="7149163576:AAGPh9YitHcQavs4qu34ZVjvmyeKU189a-4"
chat_id="1418688827"
MESSAGE="TF Modisco is done!"
url="https://api.telegram.org/bot" + TOKEN + "/sendMessage?chat_id=" + chat_id + "&text=" + MESSAGE
requests.get(url)


2. Create a single list with all patterns and trim patterns by information content

In [ ]:
pos_patterns, neg_patterns = [], []
for topic_patterns in topics_to_patterns:
        p, n = topic_patterns
        if p is not None:
            pos_patterns.extend(p)
        if n is not None:
            neg_patterns.extend(n)

all_patterns = [*pos_patterns, *neg_patterns]

trimmed_patterns = [
    workflow.trim_pattern_by_ic(pattern, 0.1) for pattern in all_patterns
]

3. Get seqlets for all regions and all tasks/topics

In [ ]:
seqlets = workflow.get_seqlets_for_all_tasks(
    one_hot = seq_onehots,
    task_to_hyp_scores = task_to_hyp_scores,
    flank_size = 10
)

# ping me
import os
import requests
TOKEN="7149163576:AAGPh9YitHcQavs4qu34ZVjvmyeKU189a-4"
chat_id="1418688827"
MESSAGE=f"Getting seqlets is done!"
url="https://api.telegram.org/bot" + TOKEN + "/sendMessage?chat_id=" + chat_id + "&text=" + MESSAGE
requests.get(url)

4. Match seqlets to identified patterns (based on similarities of the pattern)

In [ ]:
from custom_modisco import hit_scoring
match_scores_patterns, fwd_rev_pattern = hit_scoring.match_score_seqlets_patterns(
        seqlets, trimmed_patterns)

# ping me
import os
import requests
TOKEN="7149163576:AAGPh9YitHcQavs4qu34ZVjvmyeKU189a-4"
chat_id="1418688827"
MESSAGE=f"Matching seqlets to patterns is done!"
url="https://api.telegram.org/bot" + TOKEN + "/sendMessage?chat_id=" + chat_id + "&text=" + MESSAGE
requests.get(url)

5. Threshold the similarity scores

In [ ]:
import numpy as np
thresholds = np.zeros(len(trimmed_patterns))

from tqdm import tqdm
for i, pattern in tqdm(enumerate(trimmed_patterns), total = len(trimmed_patterns)):
        thresholds[i] = hit_scoring.get_sim_threshold_ROC(
                match_scores = list(match_scores_patterns[i, :, 0]),
                pattern = pattern)

6. Calculate contribution scores

In [ ]:
contrib_scores = hit_scoring.contrib_score_seqlets_pattern(
    seqlets, trimmed_patterns, match_scores_patterns)

# save
pickle.dump(
    contrib_scores,
    open(os.path.join("contrib_scores.pkl"), "wb"))

# ping me
import os
import requests
TOKEN="7149163576:AAGPh9YitHcQavs4qu34ZVjvmyeKU189a-4"
chat_id="1418688827"
MESSAGE=f"Calculation of contribution scores is done!"
url="https://api.telegram.org/bot" + TOKEN + "/sendMessage?chat_id=" + chat_id + "&text=" + MESSAGE
requests.get(url)

7. Get hits on regions

In [ ]:
df_region_hits = workflow.get_hits(
        patterns = trimmed_patterns,
        seqlets = seqlets,
        fwd_rev_pattern = fwd_rev_pattern,
        match_scores = match_scores_patterns,
        contrib_scores = contrib_scores,
        match_score_thresholds = thresholds,
        contrib_threshold_quant = 0.60)

# save
df_region_hits.to_csv(
        os.path.join("df_region_hits.tsv"),
        sep = '\t', index=False
)

Load data

In [ ]:
import os
import pickle

# load predictions
#with open(os.path.join("prediction_score.pkl"), "rb") as f:
#    prediciton_scores = pickle.load(f)

# load TFModisco patterns results
with open(os.path.join("topics_to_patterns.pkl"), "rb") as f:
    topic_to_patterns = pickle.load(f)

with open(os.path.join("contrib_scores.pkl"), "rb") as f:
    contrib_scores =  pickle.load(f)


Cluster patterns

In [ ]:
import modiscolite as modisco
import numpy as np

def flatten_list(l):
        return [item for sublist in l for item in sublist]

pos_patterns, neg_patterns = [], []
for topic_patterns in topic_to_patterns:
        p, n = topic_patterns
        if p is not None:
            pos_patterns.extend(p)
        if n is not None:
            neg_patterns.extend(n)

affmat_pos = modisco.affinitymat.jaccard_from_seqlets(
      seqlets = pos_patterns,
      min_overlap = 0.9,
      seqlet_neighbors = np.array([list(range(len(pos_patterns))) for x in pos_patterns]))
affmat_neg = modisco.affinitymat.jaccard_from_seqlets(
      seqlets = neg_patterns,
      min_overlap = 0.9,
      seqlet_neighbors = np.array([list(range(len(neg_patterns))) for x in neg_patterns]))

from modiscolite.tfmodisco import _density_adaptation
csr_density_adapted_affmat_pos = _density_adaptation(
        affmat_pos, np.array([list(range(len(pos_patterns))) for x in pos_patterns]),
        100)
csr_density_adapted_affmat_neg = _density_adaptation(
        affmat_neg, np.array([list(range(len(neg_patterns))) for x in neg_patterns]),
        100)

cluster_indices_pos = modisco.cluster.LeidenCluster(
        csr_density_adapted_affmat_pos,
        n_seeds=50,
        n_leiden_iterations=-1)
cluster_indices_neg = modisco.cluster.LeidenCluster(
        csr_density_adapted_affmat_neg,
        n_seeds=50,
        n_leiden_iterations=-1
)


Plots

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logomaker
from tqdm import tqdm

# sort positive patterns by cluster membership
cluster_indices_pos_idx = np.argsort(cluster_indices_pos)
cluster_indices_pos_sort = cluster_indices_pos[cluster_indices_pos_idx]

# plot patterns for each cluster
for cluster_idx in np.unique(cluster_indices_pos_sort):
        print(cluster_idx)
        # get members of the cluster
        cluster_indices_pos_idx_cl = cluster_indices_pos_idx[cluster_indices_pos == cluster_idx]
        # prepare figure
        fig, axs = plt.subplots(
                nrows = len(cluster_indices_pos_idx_cl),
                figsize = (8, len(cluster_indices_pos_idx_cl) * 3))
        # loop over all patterns in the cluster
        for pattern_idx, ax in tqdm(zip(cluster_indices_pos_idx_cl, axs.ravel()), total = len(pos_patterns)):
                pattern = pos_patterns[pattern_idx]
                df = pd.DataFrame(pattern.contrib_scores, columns=['A', 'C', 'G', 'T'])
                df.index.name = 'pos'
                crp_logo = logomaker.Logo(df, ax=ax)
                crp_logo.style_spines(visible=False)
                ax.get_xaxis().set_visible(False)
                ax.set_title(f'pattern_{pattern_idx}; cluster {cluster_idx}')
                ax.set_ylim(min(df.sum(axis=1).min(), 0), df.sum(axis=1).max())
        fig.tight_layout()
        fig.savefig(os.path.join(f'plots/all_patterns_pos_cluster{cluster_idx}.pdf'))

# sort negative patterns by cluster membership
cluster_indices_neg_idx = np.argsort(cluster_indices_neg)
cluster_indices_neg_sort = cluster_indices_neg[cluster_indices_neg_idx]

# plot patterns for each cluster
for cluster_idx in np.unique(cluster_indices_neg_sort):
        print(cluster_idx)
        # get members of the cluster
        cluster_indices_neg_idx_cl = cluster_indices_neg_idx[cluster_indices_neg == cluster_idx]
        # prepare figure
        fig, axs = plt.subplots(
                nrows = len(cluster_indices_neg_idx_cl),
                figsize = (8, len(cluster_indices_neg_idx_cl) * 3))
        # loop over all patterns in the cluster
        for pattern_idx, ax in tqdm(zip(cluster_indices_neg_idx_cl, axs.ravel()), total = len(pos_patterns)):
                pattern = neg_patterns[pattern_idx]
                df = pd.DataFrame(pattern.contrib_scores, columns=['A', 'C', 'G', 'T'])
                df.index.name = 'pos'
                crp_logo = logomaker.Logo(df, ax=ax)
                crp_logo.style_spines(visible=False)
                ax.get_xaxis().set_visible(False)
                ax.set_title(f'pattern_{pattern_idx}; cluster {cluster_idx}')
                ax.set_ylim(min(df.sum(axis=1).min(), 0), df.sum(axis=1).max())
        fig.tight_layout()
        fig.savefig(os.path.join(f'plots/all_patterns_neg_cluster{cluster_idx}.pdf'))


Plot all positive patterns together, and all negative patterns together. 

In [ ]:
import matplotlib.pyplot as plt
import logomaker
from tqdm import tqdm

fig, axs = plt.subplots(
        nrows = len(pos_patterns),
        figsize = (8, len(pos_patterns) * 4))

for pattern_idx, ax in tqdm(zip(np.argsort(cluster_indices_pos), axs.ravel()), total = len(pos_patterns)):
        pattern = pos_patterns[pattern_idx]
        df = pd.DataFrame(pattern.contrib_scores, columns=['A', 'C', 'G', 'T'])
        df.index.name = 'pos'
        crp_logo = logomaker.Logo(df, ax=ax)
        crp_logo.style_spines(visible=False)
        ax.get_xaxis().set_visible(False)
        ax.set_title(f'pattern_{pattern_idx}')
        ax.set_ylim(min(df.sum(axis=1).min(), 0), df.sum(axis=1).max())

fig.tight_layout()
fig.savefig(os.path.join('plots/all_patterns_pos.pdf'))

fig, axs = plt.subplots(
        nrows = len(neg_patterns),
        figsize = (8, len(neg_patterns) * 4))

for pattern_idx, ax in tqdm(zip(np.argsort(cluster_indices_neg), axs.ravel()), total = len(neg_patterns)):
        pattern = neg_patterns[pattern_idx]
        df = pd.DataFrame(pattern.contrib_scores, columns=['A', 'C', 'G', 'T'])
        df.index.name = 'pos'
        crp_logo = logomaker.Logo(df, ax=ax)
        crp_logo.style_spines(visible=False)
        ax.get_xaxis().set_visible(False)
        ax.set_title(f'pattern_{pattern_idx}')
        ax.set_ylim(min(df.sum(axis=1).min(), 0), df.sum(axis=1).max())

fig.tight_layout()
fig.savefig(os.path.join('plots/all_patterns_neg.pdf'))


Save trimmed patterns

In [ ]:
from tqdm import tqdm
from custom_modisco.workflow import trim_pattern_by_ic

background = np.array([0.30, 0.18, 0.21, 0.31])

# positive patterns
patterns = np.argsort(cluster_indices_pos)
pwm_list = []

for pattern_idx in tqdm(patterns, total = len(patterns)):
    untrimmed_pattern = pos_patterns[pattern_idx]
    try:
        trimmed_pattern = trim_pattern_by_ic(pattern = untrimmed_pattern, min_v = 0.25, background = background)
    except:
        trimmed_pattern = untrimmed_pattern
    df = pd.DataFrame(trimmed_pattern.contrib_scores, columns = ['A', 'C', 'G', 'T']).fillna(0)
    df[df < 0] = 0
    pwm_list.append(df)

# save txt file
with open(os.path.join('patterns_pos.txt'),'w') as txt_file:
    for i in range(len(pwm_list)):
        pwm=pwm_list[i]
        pwm.to_csv(txt_file, sep="\t", header=False, index=False)
        txt_file.write("\n")

# save meme file
#with open(os.path.join(tfmodisco_dir, 'patterns_pos_.meme'),'w') as meme_file:
#    meme_file.write("MEME version 5.0\n\nALPHABET= ACGT\n\nstrands: + -\n\n")
#    for i in range(len(pwm_list)):
#        pwm=pwm_list[i]
#        meme_file.write(f"MOTIF pos_pattern_{patterns[i]}\n")
#        meme_file.write(f"letter-probability matrix: alength= 4 w= {len(pwm.columns)} nsites= 20\n")
#        pwm.to_csv(meme_file, sep="\t", header=False, index=False)
#        meme_file.write("\n")

with open(os.path.join('patterns_pos.names'),'w') as name_file:
    for i in range(len(pwm_list)):
        name_file.write(f'pos_pattern_{patterns[i]}\n')

# negative patterns
patterns = np.argsort(cluster_indices_neg)
pwm_list = []

for pattern_idx in tqdm(patterns, total = len(patterns)):
    untrimmed_pattern = neg_patterns[pattern_idx]
    try:
        trimmed_pattern = trim_pattern_by_ic(pattern = untrimmed_pattern, min_v = 0.25, background = background)
    except:
        trimmed_pattern = untrimmed_pattern
    df = pd.DataFrame(trimmed_pattern.contrib_scores, columns = ['A', 'C', 'G', 'T']).fillna(0).abs()
    pwm_list.append(df)

# save txt file
with open(os.path.join('patterns_neg.txt'),'w') as txt_file:
    for i in range(len(pwm_list)):
        pwm=pwm_list[i]
        pwm.to_csv(txt_file, sep="\t", header=False, index=False)
        txt_file.write("\n")

# save meme file
#with open(os.path.join(tfmodisco_dir, 'patterns_neg_.meme'),'w') as meme_file:
#    meme_file.write("MEME version 5.0\n\nALPHABET= ACGT\n\nstrands: + -\n\n")
#    for i in range(len(pwm_list)):
#        pwm=pwm_list[i]
#        meme_file.write(f"MOTIF neg_pattern_{patterns[i]}\n")
#        meme_file.write(f"letter-probability matrix: alength= 4 w= {len(pwm.columns)} nsites= 20\n")
#        pwm.to_csv(meme_file, sep="\t", header=False, index=False)
#        meme_file.write("\n")

# save names
with open(os.path.join('patterns_neg.names'),'w') as name_file:
    for i in range(len(pwm_list)):
        name_file.write(f'neg_pattern_{patterns[i]}\n')

Match patterns to motifs

In [ ]:
motifs="/staging/leuven/stg_00002/lcb/aelek/data_nvec/motif-archetypes-PPM-PCCnorm-0.8-IC0.5-8bp-unique-pwms.meme"
id="archetypes"
tfmodisco_dir="/staging/leuven/stg_00002/lcb/aelek/gkmsvm_nvec/"

patterns=${tfmodisco_dir}"/patterns_pos.txt"
outdir=${tfmodisco_dir}"$(basename ${patterns})"
outdir=${outdir%%.txt}
scripts/tomtom.sh ${patterns} ${motifs} ${id} ${outdir}

patterns=${tfmodisco_dir}"/patterns_neg.txt"
outdir=${tfmodisco_dir}"$(basename ${patterns})"
outdir=${outdir%%.txt}
scripts/tomtom.sh ${patterns} ${motifs} ${id} ${outdir}

Calculate contribution of all patterns for all topics

In [ ]:
import sys
from custom_modisco.workflow import get_pattern_contribution_for_all_tasks

import numpy as np
task_to_hyp_scores = {task: np.array(task_to_hyp_scores[task]) for task in task_to_hyp_scores.keys()}

# calculate contribution for all patterns
selected_patterns_pos = np.argsort(cluster_indices_pos)
selected_patterns_neg = np.argsort(cluster_indices_neg)

pattern_to_task_contribution = {}

from tqdm import tqdm
for pattern_id in tqdm(np.argsort(cluster_indices_pos), total = len(cluster_indices_pos)):
        pattern = pos_patterns[pattern_id]
        name = f"pos_pattern_{pattern_id}"
        pattern_to_task_contribution[name] = get_pattern_contribution_for_all_tasks(
                pattern = pattern,
                one_hot = seq_onehots, task_to_hyp_scores = task_to_hyp_scores)

for pattern_id in tqdm(np.argsort(cluster_indices_neg), total = len(cluster_indices_neg)):
        pattern = neg_patterns[pattern_id]
        name = f"neg_pattern_{pattern_id}"
        pattern_to_task_contribution[name] = get_pattern_contribution_for_all_tasks(
                pattern = pattern,
                one_hot = seq_onehots, task_to_hyp_scores = task_to_hyp_scores)

# save
pickle.dump(
    pattern_to_task_contribution,
    open(os.path.join("pattern_to_task_contribution.pkl"), "wb"))

Make matrix with contributions for code tables

In [ ]:
import pandas as pd
from custom_modisco.visualization import plot_pattern

# contributions
with open(os.path.join("pattern_to_task_contribution.pkl"), "rb") as f:
    pattern_to_task_contribution = pickle.load(f)

df_pattern_to_task_contribution = pd.DataFrame(pattern_to_task_contribution).T

# matrix dimensions
nrows = len(df_pattern_to_task_contribution.index)
ncols = len(df_pattern_to_task_contribution.columns)

# get contributions into matrix format
df_pattern_to_task_contribution_max = pd.DataFrame(
        index = df_pattern_to_task_contribution.index,
        columns = df_pattern_to_task_contribution.columns).fillna(0)

for pattern in df_pattern_to_task_contribution_max.index:
        for task in df_pattern_to_task_contribution_max.columns:
                df_pattern_to_task_contribution_max.loc[pattern, task] = abs(
                        df_pattern_to_task_contribution.loc[pattern, task].contrib_scores
                ).max() * np.sign(df_pattern_to_task_contribution.loc[pattern, task].contrib_scores.mean())

# for each pattern, which topic it has max contribution to? to order patterns
sorted_patterns = pd.DataFrame(df_pattern_to_task_contribution_max.idxmax(axis = 1)).reset_index().set_index(0)
sorted_patterns = sorted_patterns['index'].to_list()

# order, map topics to cell types and save table
df_pattern_to_task_contribution_max = df_pattern_to_task_contribution_max.loc[sorted_patterns].copy()
df_pattern_to_task_contribution_max.to_csv(os.path.join('pattern_to_task_contribution.tsv'), sep='\t')

Code tables for selected patterns based on expression correlation (see `.Rmd`)

### TF-MoDISco

Generate dinuc shuffled sequences for computing null distribution of importance scores

In [9]:
from deeplift.dinuc_shuffle import dinuc_shuffle

import numpy as np
import random
np.random.seed(1234)
random.seed(1234)

num_dinuc_shuffled_seqs = 1000

# Generate the dinucleotide shuffled sequences
fasta_seqs_test=[]
with open(dir+model+".explain.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        fasta_seqs_test.append(str(record.seq))
        
fasta_seqs_no_N = [x.rstrip() for x in fasta_seqs_test if ('N' not in x)]

# Save
open(dir+model+".dnshuff.fasta", 'w').write(
 "\n".join([">seq"+str(i)+"\n"+dinuc_shuffle(np.random.choice(fasta_seqs_no_N).rstrip())
            for i in range(num_dinuc_shuffled_seqs)]))


51879

Load importance scores for shuffled sequences

In [ ]:
dnshuff_impscores = [np.array( [[float(z) for z in y.split(",")]
                           for y in x.rstrip().split("\t")[2].split(";")])
                     for x in open(dir+model+".explain.dnshuff.txt")]
dnshuff_perposimp = [np.sum(x,axis=-1) for x in dnshuff_impscores]

In [ ]:
from modisco.visualization import viz_sequence
from matplotlib import pyplot as plt
plt.style.use('default')
import h5py
import numpy as np
import modisco

#I sum up the null distribution importance scores at each position
# to get the perpos importance scores
# of the null distribution. I decided to use
# the unnormalized dnshuff_impscores (rather than normalized versions
# of these scores) as this made for a more stringent null distribution.
# Figuring out the best way to generate a null distribution for these scores
# is still an open problem.
dnshuff_perposimp = [np.sum(x,axis=-1) for x in dnshuff_impscores]

tfmodisco_results = modisco.tfmodisco_workflow.workflow.TfModiscoWorkflow(
    #target_seqlet_fdr=0.15,
    min_passing_windows_frac=0.03,
    max_passing_windows_frac=0.2,
    seqlets_to_patterns_factory = modisco.tfmodisco_workflow.seqlets_to_patterns.TfModiscoSeqletsToPatternsFactory(
        kmer_len=6, num_gaps=1, num_mismatches=0,
   ),
)

In [ ]:
tfmodisco_results = tfmodisco_results(
    task_names=[model],
    contrib_scores={model: impscores},
    hypothetical_contribs={model: hyp_impscores},
    one_hot=onehot_data,
    null_per_pos_scores={model: dnshuff_perposimp}
)

Save

In [ ]:
import h5py
import modisco.util
grp = h5py.File(dir+model+".results.hdf5", "w")
tfmodisco_results.save_hdf5(grp)

In [ ]:

hdf5_results = h5py.File(dir+model+".results.hdf5","r")

metacluster_names = [
    x.decode("utf-8") for x in 
    list(hdf5_results["metaclustering_results"]
         ["all_metacluster_names"][:])]
print(metacluster_names)
#metacluster_grp = (hdf5_results["metacluster_idx_to_submetacluster_results"][metacluster_name])
#print("activity pattern:",metacluster_grp["activity_pattern"][:])
#pattern = metacluster_grp["seqlets_to_patterns_result"]["patterns"][pattern_name]
#hdf5_results[][model+"_hypothetical_contribs"]["fwd"]

Visualize motifs

In [ ]:
from modisco.aggregator import TrimToBestWindowByIC

trimmer = TrimToBestWindowByIC(
    window_size=30,
    onehot_track_name=[model+"_contrib_scores"],
    bg_freq=np.array([0.25,0.25,0.25,0.25])
)

In [ ]:
vars(tfmodisco_results.multitask_seqlet_creation_results)

In [ ]:

print("num seqlets",len(pattern.seqlets))
print("fwd seq PWM")
viz_sequence.plot_weights(viz_sequence.ic_scale(
    pattern["sequence"].fwd, background=np.array([0.25,0.25,0.25,0.25])))
print("Contrib scores")
viz_sequence.plot_weights(pattern["task0_contrib_scores"].fwd)
print("Hyp contrib scores")
viz_sequence.plot_weights(pattern["task0_hypothetical_contribs"].fwd)
  

In [ ]:
for i,pattern in enumerate(trimmer(tfmodisco_results.metaclustering_results.metaclusterer
                                   .metacluster_idx_to_activity_pattern[0]
                                   .seqlets_to_patterns_result.patterns)):
    print("num seqlets",len(pattern.seqlets))
    print("fwd seq PWM")
    viz_sequence.plot_weights(viz_sequence.ic_scale(
        pattern["sequence"].fwd, background=np.array([0.25,0.25,0.25,0.25])))
    print("Contrib scores")
    viz_sequence.plot_weights(pattern["task0_contrib_scores"].fwd)
    print("Hyp contrib scores")
    viz_sequence.plot_weights(pattern["task0_hypothetical_contribs"].fwd)
  
    print("rev seq PWM")
    viz_sequence.plot_weights(viz_sequence.ic_scale(
      pattern["sequence"].rev, background=np.array([0.25,0.25,0.25,0.25])))
    print("Contrib scores")
    viz_sequence.plot_weights(pattern["task0_contrib_scores"].rev)
    print("Hyp contrib scores")
    viz_sequence.plot_weights(pattern["task0_hypothetical_contribs"].rev)